In [7]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen

import re
from konlpy.tag import Okt, Kkma, Komoran
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences

import networkx
from tqdm.autonotebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import word2vec
#from difflib import ndiff

import os
os.environ['JAVA_OPTS'] = 'Xmx8192m'

In [8]:
full_execute = True

In [9]:
df = pd.read_csv('./data/naver_movie_dataset.csv')
df.head()

,Unnamed: 0,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,0,100% 울프: 푸들이 될 순 없어,애니메이션,2020,12.24,9.50,16,유서 깊은 늑대인간 가문의 후계자 프레디는 자랑스러운 아버지의 뒤를 이어 위기에 빠...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20201208_299/1...
1,1,108명의 여자와 섹스하는 방법,코미디|드라마,2020,5.21,1.00,1,유명 시나리오 작가인 '카이바 고로'는 SNS 통해 아내 '아야코'의 외도 사실을 ...,마츠오 스즈키|나카야마 미호,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200512_12/15...
2,2,16세의 사운드트랙,코미디|멜로/로맨스,2020,7.02,7.75,4,“축하는 개뿔! 이제 나만 남았잖아 쪽팔리게!!!” - 유일한 키스 無경험자 ‘메이...,스칼렛 마샬|제임스 콜로웨이,자멜 하디쿠라|잭볼|숀 미칼레프,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200619_172/1...
3,3,1917,드라마|전쟁,2020,2.19,8.87,5891,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
4,4,1942: 언노운 배틀,전쟁,2020,7.29,8.15,78,인류 역사상 최악의 전쟁이라 일컬어지는 제2차 세계대전이 한창인 1942년. 소련군...,세르게이 자코브|이반 바타레프|알렉산드르 버카로프|표트르 로가체브|이고르 그라부조프,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200810_177/1...


In [10]:
df.drop(df[df['plot'].isnull()].index, inplace=True)

df['plot'] = df['plot'].apply(str)
plots = df['plot'].copy()#pd.Series(df['plot'])

df['plot'] = list(map(lambda x:re.sub("\‘[가-힣 ]+\’(\([가-힣 ]+\))?[^\s]*"," ",x),df['plot']))
df['plot'] = list(map(lambda x:re.sub("[가-힣]+ [가-힣]+ {0,1}\([가-힣 ]+\)[^\s]*"," ",x), df['plot']))

for idx in tqdm(df.index):
    chars1, chars2 = [],[]
    
    chars1 = list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])]))
    if chars1:
        if len(chars1) == 1:
            regex1 = ''.join(list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])])))
        else:
            regex1 = '|'.join(list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])])))
        #print(regex1)
        plots[idx] = re.sub('(%s)[^\s]'%(regex1),' ',df.loc[idx,'plot'])

    chars2 = ''.join(re.findall('[가-힣]+ [가-힣]+ {0,1}\([가-힣 ]+\)',plots[idx]))
    if chars2:
        regex2 = '|'.join(map(str.strip, re.compile('\([가-힣 ]+\)').split(chars2)[:-1]))
        regex2 += '|'+' '.join(regex2.split('|')).replace(' ','|')
        #print(regex2)
        plots[idx] = re.sub('(%s)[^\s]'%(regex2),' ',df.loc[idx,'plot'])

df['plot'] = plots

In [11]:
display(len(df['plot']))
df['plot'].head(10)

15024

0    유서 깊은 늑대인간 가문의 후계자 프레디는 자랑스러운 아버지의 뒤를 이어 위기에 빠...
1    유명 시나리오 작가인 '카이바 고로'는 SNS 통해 아내 '아야코'의 외도 사실을 ...
2    “축하는 개뿔! 이제 나만 남았잖아 쪽팔리게!!!” - 유일한 키스 無경험자   “...
3    제1차 세계대전이 한창인 1917년. 독일군에   모든 통신망이 파괴된 상황 속에서...
4    인류 역사상 최악의 전쟁이라 일컬어지는 제2차 세계대전이 한창인 1942년. 소련군...
5    러시아군과 대치 중 전열에서 낙오된   부대는 다시 아군에 합류하기 위한 과정에서 ...
6    뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...
7    219 년, 우주비행사 존과 재키는 목성의 3번째 달에 건설된 '판지아' 연료 정거...
8    유명배우 베흐나즈 자파리에게 먼 시골 마을에 사는 소녀의 영상이 전달된다. 가족들이...
9    “인생 그렇게 쉽게 끝나지 않아…” 69세 효정은 병원에서 치료를 받다가 29세의 ...
Name: plot, dtype: object

In [12]:
df['plot'] = df['plot'].str.replace('\r\xa0', ' ')
display(df[df['plot'].str.contains('\r')].index)
df['plot'] = df['plot'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .][^\s]+"," ")

Int64Index([], dtype='int64')

In [13]:
df['plot'].head(20)

0     유서 깊은 늑대인간 가문의 후계자 프레디는 자랑스러운 아버지의 뒤를 이어 위기에 빠...
1     유명 시나리오 작가인   고로    통해 아내   외도 사실을 알게 된다. 여전히 ...
2       개뿔! 이제 나만 남았잖아 쪽팔리게  - 유일한 키스       난 공식적인 멍...
3     제  세계대전이 한창인   독일군에   모든 통신망이 파괴된 상황 속에서 영국군 병...
4     인류 역사상 최악의 전쟁이라 일컬어지는 제  세계대전이 한창인   소련군은 부대원의...
5     러시아군과 대치 중 전열에서 낙오된   부대는 다시 아군에 합류하기 위한 과정에서 ...
6     뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...
7       년, 우주비행사 존과 재키는 목성의   달에 건설된   연료 정거장에서 1 년동...
8     유명배우 베흐나즈 자파리에게 먼 시골 마을에 사는 소녀의 영상이 전달된다. 가족들이...
9       그렇게 쉽게 끝나지 않아    효정은 병원에서 치료를 받다가   남자 간호조무사...
10      영화감독을 꿈꾸던 ‘ 의 과거.   우연히 만난   주인공을 제안하고, 함께 촬...
11    제   세계대전 중일전쟁 초기, 거듭된 패배로 많은 사상자를 내며 중국군이 퇴각하던...
12    여자친구   대차게 차여 충격받은 물리학 천재   사랑하는 그녀의 마음을 되돌리고 ...
13    미식축구 스타   심슨과 이혼한 니콜 심슨은 홀로 두 아이를 키우며 씩씩하게 살아가...
14    원인불명 증세의 사람들의 공격에 통제 불능에 빠진 도시. 영문도 모른 채 잠에서 깬...
15    우리는 소망을 잃어버렸다. 예수의 언약을 잊어버렸다. 하지만 최근 연구를 통해 숨겨...
16    천재 기타리스트 마키노는 공연을 찾아온 저널리스트 요코에게 첫눈에 반한다. 마키노는...
17    젊고 잘생긴 남자친구 태호와 모텔에서 정사를 나누던 유부녀 허진은 갑작스런 

In [14]:
okt, kkma, kmr = Okt(), Kkma(), Komoran()

In [15]:
from collections import Counter

In [16]:
cnt = Counter()

for plot in tqdm(df['plot']):
    for word,tag in kmr.pos(plot):
        if tag in ['NNP', 'NNG', 'VV']:
            if not cnt[(word,tag)]:
                cnt[(word,tag)] = 1
            else:
                cnt[(word,tag)] += 1

In [17]:
cnt.most_common()

[(('되', 'VV'), 26689),
 (('하', 'VV'), 13969),
 (('있', 'VV'), 12510),
 (('자신', 'NNG'), 9672),
 (('위하', 'VV'), 9623),
 (('시작', 'NNG'), 7290),
 (('사람', 'NNG'), 6208),
 (('사랑', 'NNG'), 6195),
 (('받', 'VV'), 5884),
 (('알', 'VV'), 5657),
 (('만나', 'VV'), 5116),
 (('날', 'NNG'), 4642),
 (('살', 'VV'), 4473),
 (('찾', 'VV'), 4213),
 (('친구', 'NNG'), 4167),
 (('보', 'VV'), 3947),
 (('속', 'NNG'), 3919),
 (('가', 'VV'), 3893),
 (('집', 'NNG'), 3874),
 (('사건', 'NNG'), 3526),
 (('일', 'NNG'), 3434),
 (('대하', 'VV'), 3317),
 (('빠지', 'VV'), 3112),
 (('남자', 'NNG'), 3108),
 (('떠나', 'VV'), 3021),
 (('후', 'NNG'), 2994),
 (('아버지', 'NNG'), 2822),
 (('여자', 'NNG'), 2724),
 (('오', 'VV'), 2677),
 (('말', 'NNG'), 2642),
 (('가족', 'NNG'), 2510),
 (('사실', 'NNG'), 2499),
 (('느끼', 'VV'), 2420),
 (('마음', 'NNG'), 2406),
 (('마을', 'NNG'), 2360),
 (('한편', 'NNG'), 2351),
 (('죽', 'VV'), 2240),
 (('이야기', 'NNG'), 2224),
 (('때', 'NNG'), 2212),
 (('발견', 'NNG'), 2212),
 (('삶', 'NNG'), 2200),
 (('보내', 'VV'), 2200),
 (('세상', 'NNG'), 2159),


In [18]:
cnt_df = pd.DataFrame(np.hstack(np.array(cnt.most_common())[:,0]).reshape(-1,2),
                     columns=['word','tag'])
cnt_df['cnt'] = np.array(cnt.most_common())[:,1]
cnt_df.head()

C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
C:\Users\korea_1\anaconda3\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


,word,tag,cnt
0,되,VV,26689
1,하,VV,13969
2,있,VV,12510
3,자신,NNG,9672
4,위하,VV,9623


In [13]:
from nltk.corpus import stopwords
if full_execute:
    stopwords.words('english')

In [14]:
idx_cnt=50

In [15]:
if full_execute:
    idx_cnt += 50
    cnt_df[cnt_df['tag'].str.startswith('N')][idx_cnt:idx_cnt+50]

In [16]:
cnt_df.shape

(39355, 3)

In [17]:
ridx = [4, 6, 10, 15, 25, 41, 45, 60, 61, 64, 78, 90, 91, 96, 97, 102, 107, 109, 140, 145, 160, 161, 162, 186, 188,
205, 221, 234, 249, 253, 260, 268, 271, 272, 280, 281, 282, 284, 292, 302, 304, 314, 316, 320, 324, 328, 356,
362, 379, 381, 399, 415, 416, 430, 434, 453, 462, 465, 466, 469, 479, 487, 495, 496, 498, 503, 508, 514, 516,
521, 525, 528, 551, 552, 555, 563, 569, 573, 576, 577, 583, 585, 599, 600, 603, 608, 610, 612, 618, 629, 631,
650, 655, 671, 674, 683, 690, 693, 700, 701, 714, 724, 728, 729, 732, 733, 739, 742, 745, 748, 750, 754, 756,
759, 764, 771, 778, 785, 788, 789, 795, 803, 826, 830, 843, 859, 866, 876, 881, 883, 901, 913, 931, 934, 939,
947, 956, 961, 965, 967, 972, 973, 978, 990, 1010, 1027, 1034, 1045, 1046, 1053, 1058, 1063, 1078, 1088, 1089, 1109, 1110, 1112, 1116, 1120, 1137, 1141, 1145, 1147, 1155, 1157, 1158, 1160, 1168, 1179, 1187, 1207,
1210, 1211, 1214, 1223, 1225, 1231, 1234, 1238, 1241, 1256, 1264, 1267, 1269, 1278, 1283, 1286, 1290, 1293,
1301, 1308, 1316, 1319, 1322, 1341, 1352, 1354, 1355, 1370, 1375, 1390, 1391, 1402, 1415, 1417, 1418, 1421,
1423, 1430, 1432, 1437, 1445, 1456, 1457, 1458, 1459, 1481, 1492, 1497, 1500, 1501, 1504, 1505, 1509, 1517,
1518, 1523, 1533, 1535, 1538, 1541, 1560, 1565, 1575, 1576, 1591, 1595, 1596, 1601, 1607, 1608, 1611, 1617,
1619, 1624, 1634, 1637, 1638, 1642, 1652, 1657, 1663, 1664, 1689, 1699, 1701, 1706, 1721]

In [18]:
stops_idx = cnt_df[(cnt_df['tag'].str.startswith('N')) & (cnt_df['cnt'] < 66) & (cnt_df['word'].agg(len) == 1)].index

In [19]:
print(cnt_df.loc[stops_idx][['word','tag']].index)
print(cnt_df.loc[ridx,][['word','tag']].index)

Int64Index([ 2878,  2894,  2900,  2917,  2960,  2977,  2980,  2989,  3009,
             3020,
            ...
            36437, 36859, 37221, 37492, 37569, 38497, 38646, 38876, 39019,
            39090],
           dtype='int64', length=505)
Int64Index([   4,    6,   10,   15,   25,   41,   45,   60,   61,   64,
            ...
            1642, 1652, 1657, 1663, 1664, 1689, 1699, 1701, 1706, 1721],
           dtype='int64', length=259)


In [20]:
stops = np.concatenate((cnt_df.loc[stops_idx][['word','tag']].values, cnt_df.loc[ridx,][['word','tag']].values))

In [21]:
with open('./data/total_stopwords', encoding='utf-8') as f :
    totalStop = f.readlines()

In [22]:
stopwords = list(set(totalStop))

In [23]:
if full_execute:
    result_kkma = []
    for plot in tqdm(plots):
        words = []
        for word,tag in kkma.pos(plot):
            if tag in ['NNG', 'NNP', 'VV']:
                words.append(word)
        words = (' '.join(words)).strip()
        result_kkma.append(words)

In [25]:
if full_execute:
    with open('naver_plot_kkma.nlp','w',encoding='utf-8') as f:
        f.write('\n'.join(result_kkma))

    w2v_kkma = word2vec.LineSentence('./naver_plot_kkma.nlp')
    model_kkma = word2vec.Word2Vec(w2v_kkma, size=200, window=5, hs=1, min_count=2, sg=1)
    model_kkma.save('./naver_plot_kkma.model')

In [27]:
result_kmr = []
for plot in tqdm(df['plot']):
    words = []
    for word,tag in kmr.pos(plot):
        #if tag in ['NNG', 'NNP', 'VV']:
        if tag in ['NNG', 'NNP']:
            words.append(word)
    words = (' '.join(words)).strip()
    result_kmr.append(words)

In [28]:
with open('./naver_plot_kmr.nlp','w', encoding='utf-8') as f:
    f.write('\n'.join(result_kmr))

w2v_kmr = word2vec.LineSentence('./naver_plot_kmr.nlp')
model_kmr = word2vec.Word2Vec(w2v_kmr, size=200, window=5, hs=1, min_count=10, sg=1)
model_kmr.save('./naver_plot_kmr_min10_N_only.model')

In [29]:
if full_execute:
    model_kkma = word2vec.Word2Vec.load('./naver_plot_kkma.model')
model_kmr = word2vec.Word2Vec.load('./naver_plot_kmr_min10_N_only.model')

In [30]:
if full_execute:
    word = '레이싱'
    print('most similar words to',word,'in word2vec')
    print('Kkma\t\t\t\tKomoran')
    for (w1,t1),(w2,t2) in zip(model_kkma.wv.most_similar(word,topn=20), model_kmr.wv.most_similar(word,topn=20)):
        print("%-10s\t%.4f\t\t%-10s\t%.4f"%(w1,t1,w2,t2))

most similar words to 레이싱 in word2vec
Kkma				Komoran
레이서       	0.6901		레이스       	0.7390
나스카       	0.6758		레이서       	0.7270
레이스       	0.6496		경주        	0.6675
경주        	0.6220		스트리트      	0.6099
토너먼트      	0.5804		모터        	0.6037
챔피언       	0.5735		출전        	0.6023
드리프트      	0.5636		기량        	0.6017
출전        	0.5561		도전자       	0.6003
헤비급       	0.5520		하키        	0.5997
윔블던       	0.5510		우승        	0.5942
진출권       	0.5482		상금        	0.5940
듀얼        	0.5481		우승자       	0.5929
선수권       	0.5463		배팅        	0.5909
파이널       	0.5447		연승        	0.5858
악셀        	0.5409		헤비급       	0.5835
사중창       	0.5386		포커        	0.5814
고시엔       	0.5379		결승전       	0.5771
출전자       	0.5366		배틀        	0.5638
관중석       	0.5354		챔피언       	0.5573
최강자       	0.5333		판돈        	0.5550


In [31]:
class RawTaggerReader:
    def __init__(self, plot, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.plot = plot
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        ch = self.rgxSplitter.split(self.plot)
        if len(ch) == 1: ch.append('.')
        for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
            if not s: continue
            yield self.tagger.pos(s)

class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0

    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])

    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()

        if int(len(ranks) * ratio) > 10:
            cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        else:
            cand = sorted(ranks, key=ranks.get, reverse=True)[:10]
        pairness = {}
        startOf = {}
        tuples = {}

        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)

        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if len(k) == 1:
                if used.intersection(set(k)): continue
                both[k] = tuples[k]
                for w in k:
                    used.add(w)
        
        return both

In [32]:
results = []
for idx in tqdm(df.index):
    keywords = []
    tr = TextRank(window=5, coef=1)
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV'),
                   ('위하', 'VV')]).union(set(map(lambda x: tuple(x),stops)))
    tr.load(RawTaggerReader(df.loc[idx,'plot']),
            lambda w:w not in stopword and w[0] not in stopwords and
            w[0] in model_kmr.wv.vocab and (w[1] in ('NNG', 'NNP')))
            #and w[0] in model_kmr.wv.vocab and (w[1] in ('NNG', 'NNP', 'VV')))
    tr.build()
    kw = tr.extract(0.2)
    for k in sorted(kw, key=kw.get, reverse=True):
        keywords.append(k[0][0])
        #print("%s\t%g" % (k, kw[k]))
    if idx %100 == 0:
        print(idx,'/10255',keywords)
    results.append(keywords)

0 /10255 ['변신', '자랑', '가문', '아버지', '늑대인간', '후계자', '위기', '임무', '프레디', '뒤']
100 /10255 ['박사', '로저', '살상', '나치', '대위', '부대', '납치', '루마니아', '무기', '연합군']
200 /10255 ['각자', '순간', '카페', '날', '무표정', '한', '감독', '불투명', '연인', '인']
300 /10255 ['바이러스', '여성', '범죄자', '수감자', '안', '시작', '잘못', '북', '대서양', '레오나르도']
400 /10255 ['여자', '때', '아버지', '여자친구', '결혼', '자신', '집', '낯선', '동생', '자리']
500 /10255 ['경', '행사', '마련', '아버지', '무명', '각종', '일', '소동', '입원', '장례']
600 /10255 ['마음', '곳', '날', '낯선', '소녀', '매력', '고립', '기억']
700 /10255 ['악령', '골', '살인', '수사관', '축제', '여자친구', '카', '마을', '날', '힘']
800 /10255 ['사샤', '환청', '이후', '환영', '대학생', '커플', '엘리엇', '동전', '말', '값']
900 /10255 ['말', '해고', '정환', '대표', '사전', '전과자', '난생처음', '의', '학비', '아들']
1000 /10255 ['야생', '해결', '호기심', '날', '완성', '송', '단', '돌이', '곰', '팬더']
1100 /10255 ['의지', '북극', '속', '무덤', '추락', '기지', '헬기', '사고', '이후', '경계']
1200 /10255 ['독고', '다이', '요양', '사인', '유산', '돈', '앞', '살인', '게임', '시작']
1300 /10255 ['돈', '자신', '석', '잠자리', '친구', '부부', '관계', '종', '아이', '아내']


11200 /10255 ['선조', '사무라이', '아래', '실력', '게', '질주', '자랑', '의문', '사건', '분위기']
11300 /10255 ['사고방식', '뉴잉글랜드', '리에', '자신', '여자', '인생', '맥', '연애', '선율', '학기']
11400 /10255 ['시티', '베이', '을', '안위', '범죄자', '어머니', '판매', '지하', '수사', '증거', '부유', '사업가', '시와', '파트너']
11500 /10255 ['모습', '아빠', '해녀', '마음', '현재', '맹활약', '스무살', '꿈', '생각', '영']
11600 /10255 ['궁', '복', '수리', '청해', '해적', '검술', '정화', '위기', '공', '창', '자신', '도움', '정년', '당나라', '대결']
11700 /10255 ['야쿠자', '자신', '교포', '신세', '감정', '조직', '사랑', '인생', '재일', '마약', '노래', '돈', '일본']
11800 /10255 ['결혼', '어머니', '장모', '자리', '한편', '핀치', '장소', '결혼식', '준비', '말']
11900 /10255 ['살인', '사건', '경찰서', '로이', '사건', '수사', '엽기', '발생', '쿠', '조사']
12000 /10255 ['요원', '우주', '양자', '터널', '스타', '지안', '범죄자', '호송', '도중', '지옥']
12100 /10255 ['주식회사', '성격', '사건', '릴', '라드', '멤버', '서로', '게', '해결', '발생']
12200 /10255 ['비밀', '최고', '장악', '린', '백만장자', '여자', '운전사', '개인', '요원', '첩보']
12300 /10255 ['명령', '조직', '수뇌부', '명동', '사립', '고교', '입학', '파', '한국', '형님']
12400 /10255 ['친구', '케빈', '최고'

In [33]:
keyword_df = pd.DataFrame([(idx,result) for idx,result in zip(df.index,results)],columns=['idx','keywords'])

In [34]:
keyword_df.to_csv('./movie_keywords_1231.csv',index=False, encoding='utf-8')

In [37]:
len(keyword_df)

15024

In [2]:
keyword_df = pd.read_csv('./data/movie_keywords_1231.csv')

In [3]:
df = pd.read_csv('./data/naver_movie_dataset.csv')

In [4]:
merged = pd.merge(df,keyword_df,left_on=df.index,right_on=keyword_df.idx,)

In [5]:
merged.drop(columns=['key_0', 'idx'], inplace=True)

In [6]:
merged.to_csv('./merged.csv',index=False)

In [33]:
kkma = []
with open('./naver_plot_kkma.nlp', 'r', encoding='utf8') as f :
    for line in f :
        kkma.append(line)
kkma

['유서 늑대 인간 가문 후계자 자랑 아버지 뒤 잇 위기 빠지 인간 구하 임무 수행 날 손꼽 기다리 꿈 덜 변신 순간 늑대 푸들 되 레 디 자신 순도 늑대 인간 증명 위하 가문 비밀 담기 보물 문 스톤 찾 나서\n',
 '유명 시나리오 작가 카 고 늘 통하 아내 아 코 의 사실 알 되 아내 사랑 있 솟구치 화 참으 카 고 기상천외 복수 방법 고민 아내 게시물 가 눌리 영감 받 달 동안 여자 만나 결심\n',
 '축하 개뿔 남 쪽팔리 유 일한 키스 경험자 낳 공식적 멍청이 자칭 등급 현실 등급 바 현생 탈출 원하 인생 별것 아니 일 로 별일 하 보 전지 시점 반짝이 흑 역사 모\n',
 '세계 대전 한창 독일군 모든 통신망 파괴 상황 속 영국군 병사 필드 조지 맥 케이 오 블레이크 디 채프먼 하나 미션 주어지 함정 빠지 영국군 부대 수장 매켄지 중령 베네 배치 장군 콜린 의 공격 중지 명령 전하 둘 아군 블레이크 구하 위하 전쟁터 한복판 가로지르 사투 잇 갈\n',
 '인류 역 사상 최악 전쟁 일컫 세계 대전 한창 소련군 부대원 잃 처절 전투 끝 군사적 요충지 마을 점령 성공 마을 재탈환하 위하 독일군 공격 거세 지지 보 지원 고사하 부대 전체 전멸 위치 사 수하 이해 명령 주어지 기억 승리 만들 내야 하\n',
 '러시아 군 대치 전열 낙오 부대 아군 합류 위하 과정 러시아 의무병 생포 되 적과 불편 동행 하 되 부하 불만 쌓이 가게 되 사로잡히 러시아 의무병 탈출 위하 호시탐탐 기회 노리\n',
 '뉴욕 맨해튼 중심 벌어지 경찰 연쇄 살해 사건 범인 잡 위하 베테랑 경찰 극단 조치 취하 지금 뉴욕 맨해튼 출구 전면 봉쇄 주어지 시간 단 시간 모든 수단 방법 동원하 놈 잡 하\n',
 '우주 비행사 존 재키 목성 달 건설 판지 아 연료 정거장 동안 근무 중이 그러 날 지구 폭발 흔적 사라지 하루 아침 최후 인류 되 이 우주 가운데 인류 희망 위하 마지막 사투 벌이\n',
 '유명 배우 베 나 자 파리 시골 마을 살 소녀 영상 전달 가족 배우 되 자신 꿈 반대 

In [34]:
Kkma = pd.DataFrame({"plot_preprocessed_kkma" : kkma})

In [35]:
Kkma

,plot_preprocessed_kkma
0,유서 늑대 인간 가문 후계자 자랑 아버지 뒤 잇 위기 빠지 인간 구하 임무 수행 날...
1,유명 시나리오 작가 카 고 늘 통하 아내 아 코 의 사실 알 되 아내 사랑 있 솟구...
2,축하 개뿔 남 쪽팔리 유 일한 키스 경험자 낳 공식적 멍청이 자칭 등급 현실 등급 ...
3,세계 대전 한창 독일군 모든 통신망 파괴 상황 속 영국군 병사 필드 조지 맥 케이 ...
4,인류 역 사상 최악 전쟁 일컫 세계 대전 한창 소련군 부대원 잃 처절 전투 끝 군사...
...,...
15019,스페인 비 사 투우사 지망생 후안 가야 르 크리 스토퍼 라이 늘 도살장 황소 들어오...
15020,실업가 파파 조 글루 성적 호기심 강하 남자 낳 여배우 처제 일광욕 광경 보 위하 ...
15021,필 립 바람 이지만 아내 메이 사랑 메이 사업 핑계 남편 불만 그러 날 테니스장 메...
15022,금광 찾 알 카 오 리 살인범 블랙 오두막 금광 찾 만나 산속 갖히 이 원조 청하 ...


In [36]:
dataframe = pd.read_csv('./data/merged.csv')

In [38]:
merged = pd.merge(dataframe, Kkma, left_on=dataframe.index, right_on=Kkma.index,)

In [39]:
merged

,key_0,Unnamed: 0,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url,keywords,plot_preprocessed_kkma
0,0,0,100% 울프: 푸들이 될 순 없어,애니메이션,2020,12.24,9.50,16,유서 깊은 늑대인간 가문의 후계자 프레디는 자랑스러운 아버지의 뒤를 이어 위기에 빠...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20201208_299/1...,"['변신', '자랑', '가문', '아버지', '늑대인간', '후계자', '위기',...",유서 늑대 인간 가문 후계자 자랑 아버지 뒤 잇 위기 빠지 인간 구하 임무 수행 날...
1,1,1,108명의 여자와 섹스하는 방법,코미디|드라마,2020,5.21,1.00,1,유명 시나리오 작가인 '카이바 고로'는 SNS 통해 아내 '아야코'의 외도 사실을 ...,마츠오 스즈키|나카야마 미호,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200512_12/15...,"['달', '여자', '복수', '고민', '기상천외', '방법', '아내', '유...",유명 시나리오 작가 카 고 늘 통하 아내 아 코 의 사실 알 되 아내 사랑 있 솟구...
2,2,2,16세의 사운드트랙,코미디|멜로/로맨스,2020,7.02,7.75,4,“축하는 개뿔! 이제 나만 남았잖아 쪽팔리게!!!” - 유일한 키스 無경험자 ‘메이...,스칼렛 마샬|제임스 콜로웨이,자멜 하디쿠라|잭볼|숀 미칼레프,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200619_172/1...,[],축하 개뿔 남 쪽팔리 유 일한 키스 경험자 낳 공식적 멍청이 자칭 등급 현실 등급 ...
3,3,3,1917,드라마|전쟁,2020,2.19,8.87,5891,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...,"['버', '배치', '장군', '속', '중령', '독일군', '맥', '케이',...",세계 대전 한창 독일군 모든 통신망 파괴 상황 속 영국군 병사 필드 조지 맥 케이 ...
4,4,4,1942: 언노운 배틀,전쟁,2020,7.29,8.15,78,인류 역사상 최악의 전쟁이라 일컬어지는 제2차 세계대전이 한창인 1942년. 소련군...,세르게이 자코브|이반 바타레프|알렉산드르 버카로프|표트르 로가체브|이고르 그라부조프,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200810_177/1...,"['전멸', '거세', '한창', '소련군', '독일군', '전체', '위치', '...",인류 역 사상 최악 전쟁 일컫 세계 대전 한창 소련군 부대원 잃 처절 전투 끝 군사...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15019,15019,15023,혈과 사,드라마|멜로/로맨스,1990,6.09,8.86,7,스페인의 세빌랴에 사는 투우사 지망생 후안 가야르도(Ju n: 크리스토퍼 라이들 분...,크리스토퍼 라이델|샤론 스톤|아나 토렌트,길레르모 몬데시노스|앨버트 비달|시몬 앤드루|안토니오 곤잘레스 플로레스|호세 루이스...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/static/movie/2012/06/d...,"['후안', '투우', '노', '투우사', '카르멘', '세라', '장', '프로...",스페인 비 사 투우사 지망생 후안 가야 르 크리 스토퍼 라이 늘 도살장 황소 들어오...
15020,15020,15024,호기심에 대하여,드라마,1990,7.14,6.50,2,52세의 유복한 실업가 죠지 파파 조글루는 성적 호기심이 퍽 강한 남자다. 어느 날...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/static/movie/2012/06/d...,"['처제', '위', '아줌마', '피신', '호기심', '조', '니스', '불능...",실업가 파파 조 글루 성적 호기심 강하 남자 낳 여배우 처제 일광욕 광경 보 위하 ...
15021,15021,15025,홍콩연가,멜로/로맨스|드라마,1990,1.26,4.00,1,필립은 비록 바람둥이지만 아내 메이만은 끔찍히 사랑하나. 하지만 메이는 사업핑계로 ...,주윤발|왕소봉|정문아,우양|황진선|서숙원|주보산|백문표|양해광|나란,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20111212_56/13...,"['메이', '피터', '아내', '필립', '남편', '바람둥이', '지만', '...",필 립 바람 이지만 아내 메이 사랑 메이 사업 핑계 남편 불만 그러 날 테니스장 메...
15022,15022,15026,황금광시대,코미디|가족|모험,1990,2.17,9.04,141,금광을 찾아 알래스카에 온 찰리는 살인범 블랙의 오두막에서 금광을 찾았다는 멕케이를...,찰리 채플린|맥 스웨인|톰 머레이|헨리 버그맨,앨버트 오스틴|체스터 콘클린|베티 모리시|타이니 샌포드|밥 켈리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20111212_135/1...,"['찰리', '배', '금광', '카메라', '앞', '발견', '조지아', '임'...",금광 찾 알 카 오 리 살인범 블랙 오두막 금광 찾 만나 산속 갖히 이 원조 청하 ...


In [40]:
merged.to_csv('./merged.csv',index=False)